In [1]:
from CNN.CNN import MultivariateCNN
from models_utils.Datasets import *
from models_utils.GLOBALS import *
import torch.nn.functional as F

from CNN.cnn_utils import get_train_data, train_1D_cnn

In [2]:
# get train data
train_data = pd.read_csv('csv/train.csv', index_col=0)
train_data['activity'] = train_data['activity'].map(activity_id_mapping)
data_type_1 = train_data[train_data['sensor'] == 'vicon'].reset_index()
data_type_2 = train_data[train_data['sensor'] == 'smartwatch'].reset_index()

In [3]:
# sizes of padding/cutting
target_size_type1 = 3000
target_size_type2 = 1169

In [4]:
# train or load models
train_or_load_autoencoders = 'train'
if train_or_load_autoencoders == 'train':
    model_CNN_type1 = train_1D_cnn('Type1Only1DCNN', data_type_1, '1', target_size_type1, 30, 64, 0.001)
    model_CNN_type2 = train_1D_cnn('Type2Only1DCNN', data_type_2, '2', target_size_type2, 20, 64, 0.001)
elif train_or_load_autoencoders == 'load':
    model_CNN_type1 = MultivariateCNN(3, target_size_type1, 18).to(device)
    model_CNN_type1.load_state_dict(torch.load('Type1Only1DCNN.pth'))
    model_CNN_type2 = MultivariateCNN(3, target_size_type2, 18).to(device)
    model_CNN_type2.load_state_dict(torch.load(f'Type2Only1DCNN.pth'))
else:
    raise ValueError('Wrong train or load')

--------------
Saving Best Model with loss: 1.8948339088396593
Epoch [1/30], Epoch Duration: 30.69 seconds
Epoch [1/30], Training Loss: 2.8453, Training Accuracy: 15.38%
Epoch [1/30], Validation Loss: 1.8948, Validation Accuracy: 19.13%
--------------
Saving Best Model with loss: 1.8002430417320945
Epoch [2/30], Epoch Duration: 29.43 seconds
Epoch [2/30], Training Loss: 1.8566, Training Accuracy: 22.85%
Epoch [2/30], Validation Loss: 1.8002, Validation Accuracy: 22.82%
--------------
Saving Best Model with loss: 1.745712551203641
Epoch [3/30], Epoch Duration: 29.64 seconds
Epoch [3/30], Training Loss: 1.7717, Training Accuracy: 26.70%
Epoch [3/30], Validation Loss: 1.7457, Validation Accuracy: 27.27%
--------------
Saving Best Model with loss: 1.67983985489065
Epoch [4/30], Epoch Duration: 29.39 seconds
Epoch [4/30], Training Loss: 1.7103, Training Accuracy: 29.22%
Epoch [4/30], Validation Loss: 1.6798, Validation Accuracy: 28.90%
--------------
Saving Best Model with loss: 1.665573908

RuntimeError: Error(s) in loading state_dict for MultivariateCNN:
	size mismatch for conv1.weight: copying a param with shape torch.Size([64, 1, 3]) from checkpoint, the shape in current model is torch.Size([64, 3, 3]).
	size mismatch for fc1.weight: copying a param with shape torch.Size([512, 288000]) from checkpoint, the shape in current model is torch.Size([512, 96000]).

In [5]:
model_CNN_type2 = train_1D_cnn('Type2Only1DCNN', data_type_2, '2', target_size_type2, 20, 64, 0.001)

--------------
Saving Best Model with loss: 0.7674008317683872
Epoch [1/20], Epoch Duration: 295.83 seconds
Epoch [1/20], Training Loss: 1.2405, Training Accuracy: 56.91%
Epoch [1/20], Validation Loss: 0.7674, Validation Accuracy: 71.75%
--------------
Saving Best Model with loss: 0.5778459253018362
Epoch [2/20], Epoch Duration: 130.98 seconds
Epoch [2/20], Training Loss: 0.5090, Training Accuracy: 81.39%
Epoch [2/20], Validation Loss: 0.5778, Validation Accuracy: 80.51%
--------------
Saving Best Model with loss: 0.4035792191300476
Epoch [3/20], Epoch Duration: 130.61 seconds
Epoch [3/20], Training Loss: 0.2538, Training Accuracy: 91.34%
Epoch [3/20], Validation Loss: 0.4036, Validation Accuracy: 88.37%
--------------
Epoch [4/20], Epoch Duration: 128.41 seconds
Epoch [4/20], Training Loss: 0.1417, Training Accuracy: 95.24%
Epoch [4/20], Validation Loss: 0.4317, Validation Accuracy: 89.25%
--------------
Saving Best Model with loss: 0.3591478703576222
Epoch [5/20], Epoch Duration: 136

RuntimeError: Error(s) in loading state_dict for MultivariateCNN:
	size mismatch for conv1.weight: copying a param with shape torch.Size([64, 1, 3]) from checkpoint, the shape in current model is torch.Size([64, 3, 3]).
	size mismatch for fc1.weight: copying a param with shape torch.Size([512, 112128]) from checkpoint, the shape in current model is torch.Size([512, 37376]).

In [5]:
# calculate or load train_data
embedding_names = [f'embedding_feature_{i + 1}' for i in range(18)]
calculate_or_load_train_data = 'load'
if calculate_or_load_train_data == 'calculate':
    data_type_1, data_type_2 = get_train_data(['train_Only_CNN'], model_CNN_type1, model_CNN_type2,
                                              embedding_size=18, is_autoencoder=False)
elif calculate_or_load_train_data == 'load':
    data_type_1 = pd.read_csv('train_Only_CNN_type1.csv')
    data_type_2 = pd.read_csv('train_Only_CNN_type2.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train_Only_CNN_type1.csv'

In [39]:
# get and save results for test data
results_list = []
for i, file_id in enumerate(pd.read_csv('sample_submission.csv')['sample_id'].to_list()):

    class_path = os.path.join(files_directory, f"{file_id}.csv")
    new_data = pd.read_csv(class_path)

    if new_data.shape[1] == 3:
        if len(new_data) < 4000:
            new_data = pad_sequence(new_data, 4000)
        new_data = torch.tensor(new_data.values, dtype=torch.float32).to(device)
        normalized_new_data = (new_data - min_values_type1) / (max_values_type1 - min_values_type1 + 1e-6)
        normalized_new_data = normalized_new_data.transpose(0, 1)
        logits = model_CNN_type1(normalized_new_data)


    else:
        new_data = new_data[new_data.iloc[:, 0] == 'acceleration [m/s/s]'].iloc[:, 1:]
        if len(new_data) < 1350:
            new_data = pad_sequence(new_data, 1350)
        new_data = torch.tensor(new_data.values, dtype=torch.float32).to(device)
        normalized_new_data = (new_data - min_values_type2) / (max_values_type2 - min_values_type2 + 1e-6)
        normalized_new_data = normalized_new_data.transpose(0, 1)

        logits = model_CNN_type2(normalized_new_data)

    predictions = F.softmax(logits, dim=1)
    res_dict = {activity: predictions.squeeze()[id].item() for id, activity in id_activity_mapping.items()}

    result_dict = {label: res_dict.get(label, 0) for label in activity_id_mapping.keys()}
    result_dict['sample_id'] = file_id
    results_list.append(result_dict)
results = pd.DataFrame(results_list, columns=['sample_id'] + list(activity_id_mapping.keys()))
results.fillna(0).to_csv('results_raw_cnn.csv', index=False)